In [ ]:
import pandas as pd
import numpy as np
import os
import ast
from transformers import AutoTokenizer

In [ ]:
#coments
comments = pd.read_csv('/Users/isavalladolid/Documents/Documentos - MacBook Air de Isabel/IMEDIA_Project/data/interim/comments_data.csv')
comments

,author,body,user_subscriptions,subreddit
0,smalby,"Hell, not purgatory","['csharp', 'thefighterandthekid', 'redneckengi...",Whatcouldgowrong
1,Straight_Board_981,This gif is old enough for that baby's baby to...,['CryptidDogs'],Whatcouldgowrong
2,Consistent_Future03,What's the motto? Sending em to heaven before ...,[],Whatcouldgowrong
3,karebearjedi,'Note to self - don't drink anything when you ...,"['Austin', 'WalgreensStores']",Whatcouldgowrong
4,Calciumee,"If he is dropping them and not diddling them, ...","['Patriots', 'CasualUK', '49ers', 'chicagobull...",Whatcouldgowrong
...,...,...,...,...
8695,poeschmoe,Many or most of the main characters have cheat...,[],DunderMifflin
8696,NoHope0149,"Ok, liberal then if there's no female characte...","['canconfirmiamindian', 'GATEtard', 'NormMacdo...",DunderMifflin
8697,pmgoldenretrievers,The thing is that it’s a half second joke that...,"['thelongdark', 'explainlikeimfive', 'SierraNe...",DunderMifflin
8698,HoodGyno,god i hate phyllis.,"['NotADragQueen', 'Hyte', 'Hiphopcirclejerk']",DunderMifflin


In [57]:
#conversion a minusculas de las columnas
comments['author'] = comments['author'].str.lower()
comments['body'] = comments['body'].str.lower()
comments['subreddit'] = comments['subreddit'].str.lower()

In [58]:
#calcular por centaje de listas vacías
empty_lists_count = comments['user_subscriptions'].apply(lambda x: x == '[]').sum()
total_rows = len(comments)
empty_lists_percentage = (empty_lists_count / total_rows) * 100

print(f"Porcentaje de listas vacías: {empty_lists_percentage:.2f}%")
print(f"Cantidad de listas vacías: {empty_lists_count}")

Porcentaje de listas vacías: 10.16%
Cantidad de listas vacías: 884


In [74]:
# convertir los strings a listas
comments["user_subscriptions"] = comments["user_subscriptions"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# crear una columna que indique si el usuario está suscrito al subreddit donde hizo el comentario
comments["is_subscribed"] = comments.apply(lambda row: row["subreddit"] in row["user_subscriptions"], axis=1)

In [63]:
#calcular porcentaje de usuarios suscritos al subreddit donde comentan
porcentaje = comments["is_subscribed"].mean() * 100
print(f"Porcentaje de usuarios que comentan en subreddits a los que están suscritos: {porcentaje:.2f}%")

Porcentaje de usuarios que comentan en subreddits a los que están suscritos: 5.57%


In [64]:
comments

,author,body,user_subscriptions,subreddit,is_subscribed
0,smalby,"hell, not purgatory","[csharp, thefighterandthekid, redneckengineering]",whatcouldgowrong,False
1,straight_board_981,this gif is old enough for that baby's baby to...,[CryptidDogs],whatcouldgowrong,False
2,consistent_future03,what's the motto? sending em to heaven before ...,[],whatcouldgowrong,False
3,karebearjedi,'note to self - don't drink anything when you ...,"[Austin, WalgreensStores]",whatcouldgowrong,False
4,calciumee,"if he is dropping them and not diddling them, ...","[Patriots, CasualUK, 49ers, chicagobulls, Live...",whatcouldgowrong,False
...,...,...,...,...,...
8695,poeschmoe,many or most of the main characters have cheat...,[],dundermifflin,False
8696,nohope0149,"ok, liberal then if there's no female characte...","[canconfirmiamindian, GATEtard, NormMacdonald,...",dundermifflin,False
8697,pmgoldenretrievers,the thing is that it’s a half second joke that...,"[thelongdark, explainlikeimfive, SierraNevada,...",dundermifflin,False
8698,hoodgyno,god i hate phyllis.,"[NotADragQueen, Hyte, Hiphopcirclejerk]",dundermifflin,False


In [65]:
# cargar tokenizador de 'huggingface' 
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [67]:
# Función segura para tokenizar texto
def safe_tokenize(text):
    if isinstance(text, str):
        return tokenizer.encode(text, add_special_tokens=True)
    return []

# tokenizar columnas 'author', 'body' y 'subreddit
comments['body_tokenized'] = comments['body'].apply(safe_tokenize)
comments['subreddit_tokenized'] = comments['subreddit'].apply(safe_tokenize)


In [69]:
# Tokenizar listas de texto en 'user_subscriptions'
comments['user_subscriptions_tokenized'] = comments['user_subscriptions'].apply(
    lambda subs: [safe_tokenize(sub) for sub in subs] if isinstance(subs, list) else []
)

In [70]:
comments

,author,body,user_subscriptions,subreddit,is_subscribed,author_tokenized,body_tokenized,subreddit_tokenized,user_subscriptions_tokenized
0,smalby,"hell, not purgatory","[csharp, thefighterandthekid, redneckengineering]",whatcouldgowrong,False,"[101, 15488, 2389, 3762, 102]","[101, 3109, 1010, 2025, 16405, 28921, 7062, 102]","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]","[[101, 20116, 8167, 2361, 102], [101, 1996, 20..."
1,straight_board_981,this gif is old enough for that baby's baby to...,[CryptidDogs],whatcouldgowrong,False,"[101, 3442, 1035, 2604, 1035, 5818, 2487, 102]","[101, 2023, 21025, 2546, 2003, 2214, 2438, 200...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]","[[101, 19888, 3593, 16168, 2015, 102]]"
2,consistent_future03,what's the motto? sending em to heaven before ...,[],whatcouldgowrong,False,"[101, 8335, 1035, 2925, 2692, 2509, 102]","[101, 2054, 1005, 1055, 1996, 12652, 1029, 601...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]",[]
3,karebearjedi,'note to self - don't drink anything when you ...,"[Austin, WalgreensStores]",whatcouldgowrong,False,"[101, 10556, 2890, 4783, 2906, 6460, 4305, 102]","[101, 1005, 3602, 2000, 2969, 1011, 2123, 1005...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]","[[101, 5899, 102], [101, 24547, 28637, 3619, 2..."
4,calciumee,"if he is dropping them and not diddling them, ...","[Patriots, CasualUK, 49ers, chicagobulls, Live...",whatcouldgowrong,False,"[101, 13853, 4402, 102]","[101, 2065, 2002, 2003, 7510, 2068, 1998, 2025...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]","[[101, 11579, 102], [101, 10017, 6968, 102], [..."
...,...,...,...,...,...,...,...,...,...
8695,poeschmoe,many or most of the main characters have cheat...,[],dundermifflin,False,"[101, 18922, 11624, 5302, 2063, 102]","[101, 2116, 2030, 2087, 1997, 1996, 2364, 3494...","[101, 24654, 4063, 4328, 4246, 4115, 102]",[]
8696,nohope0149,"ok, liberal then if there's no female characte...","[canconfirmiamindian, GATEtard, NormMacdonald,...",dundermifflin,False,"[101, 2053, 26441, 24096, 26224, 102]","[101, 7929, 1010, 4314, 2059, 2065, 2045, 1005...","[101, 24654, 4063, 4328, 4246, 4115, 102]","[[101, 2064, 8663, 27972, 25107, 22254, 2937, ..."
8697,pmgoldenretrievers,the thing is that it’s a half second joke that...,"[thelongdark, explainlikeimfive, SierraNevada,...",dundermifflin,False,"[101, 7610, 21270, 2368, 13465, 7373, 14028, 102]","[101, 1996, 2518, 2003, 2008, 2009, 1521, 1055...","[101, 24654, 4063, 4328, 4246, 4115, 102]","[[101, 1996, 10052, 7662, 2243, 102], [101, 48..."
8698,hoodgyno,god i hate phyllis.,"[NotADragQueen, Hyte, Hiphopcirclejerk]",dundermifflin,False,"[101, 7415, 6292, 3630, 102]","[101, 2643, 1045, 5223, 20328, 1012, 102]","[101, 24654, 4063, 4328, 4246, 4115, 102]","[[101, 2025, 4215, 29181, 4226, 2368, 102], [1..."


In [38]:
#posts

posts = pd.read_csv('/Users/isavalladolid/Documents/Documentos - MacBook Air de Isabel/IMEDIA_Project/data/interim/posts_relacionados_data.csv')
posts

,title,score,url,subreddit
0,WCGW Showing your friend r/whatcouldgowrong,40383,https://v.redd.it/pr14bxllns2b1,Whatcouldgowrong
1,Why did she think this was a good idea?? (X-po...,3031,https://v.redd.it/4e64vi224fw21,Whatcouldgowrong
2,WCGW?,5304,https://v.redd.it/ykgbencmn29e1,Whatcouldgowrong
3,WCGW Throw a broken missile,14973,https://v.redd.it/vbg3cg30xhxd1,Whatcouldgowrong
4,Wcgw. seriously WCGW,21651,https://v.redd.it/rsmddfzertve1,Whatcouldgowrong
...,...,...,...,...
8995,Songs of Dunder Mifflin,1093,https://i.redd.it/e8snue9y32pb1.jpg,DunderMifflin
8996,John Krasinski was in my neck of the woods sco...,8587,https://i.redd.it/j18kbfv0fpv21.jpg,DunderMifflin
8997,Can someone tell me why there is a cat in a ca...,353,https://i.redd.it/96rxa6197abe1.jpeg,DunderMifflin
8998,Me trying to get everyone from r/dundermifflin...,5437,https://i.redd.it/nudq1qphve491.gif,DunderMifflin


In [71]:
posts['title'] = posts['title'].str.lower()
posts['url'] = posts['url'].str.lower()
posts['subreddit'] = posts['subreddit'].str.lower()

In [ ]:
# Función segura para tokenizar texto
def safe_tokenize(text):
    if isinstance(text, str):
        return tokenizer.encode(text, add_special_tokens=True)
    return []

# tokenizar columnas 'tittle' y 'subreddit
posts['title_tokenized'] = posts['title'].apply(safe_tokenize)
posts['subreddit_tokenized'] = posts['subreddit'].apply(safe_tokenize)

In [82]:
posts

,title,score,url,subreddit,title_tokenized,subreddit_tokenized,is_subscribed
0,wcgw showing your friend r/whatcouldgowrong,40383,https://v.redd.it/pr14bxllns2b1,whatcouldgowrong,"[101, 15868, 2290, 2860, 4760, 2115, 2767, 105...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]",False
1,why did she think this was a good idea?? (x-po...,3031,https://v.redd.it/4e64vi224fw21,whatcouldgowrong,"[101, 2339, 2106, 2016, 2228, 2023, 2001, 1037...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]",False
2,wcgw?,5304,https://v.redd.it/ykgbencmn29e1,whatcouldgowrong,"[101, 15868, 2290, 2860, 1029, 102]","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]",False
3,wcgw throw a broken missile,14973,https://v.redd.it/vbg3cg30xhxd1,whatcouldgowrong,"[101, 15868, 2290, 2860, 5466, 1037, 3714, 742...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]",False
4,wcgw. seriously wcgw,21651,https://v.redd.it/rsmddfzertve1,whatcouldgowrong,"[101, 15868, 2290, 2860, 1012, 5667, 15868, 22...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]",False
...,...,...,...,...,...,...,...
8995,songs of dunder mifflin,1093,https://i.redd.it/e8snue9y32pb1.jpg,dundermifflin,"[101, 2774, 1997, 24654, 4063, 2771, 4246, 411...","[101, 24654, 4063, 4328, 4246, 4115, 102]",NaN
8996,john krasinski was in my neck of the woods sco...,8587,https://i.redd.it/j18kbfv0fpv21.jpg,dundermifflin,"[101, 2198, 1047, 8180, 19880, 2001, 1999, 202...","[101, 24654, 4063, 4328, 4246, 4115, 102]",NaN
8997,can someone tell me why there is a cat in a ca...,353,https://i.redd.it/96rxa6197abe1.jpeg,dundermifflin,"[101, 2064, 2619, 2425, 2033, 2339, 2045, 2003...","[101, 24654, 4063, 4328, 4246, 4115, 102]",NaN
8998,me trying to get everyone from r/dundermifflin...,5437,https://i.redd.it/nudq1qphve491.gif,dundermifflin,"[101, 2033, 2667, 2000, 2131, 3071, 2013, 1054...","[101, 24654, 4063, 4328, 4246, 4115, 102]",NaN


In [78]:
#subreddit
subreddit = pd.read_csv('/Users/isavalladolid/Documents/Documentos - MacBook Air de Isabel/IMEDIA_Project/data/interim/subreddit_info_data.csv')
subreddit

,name,subscribers,description,subreddit
0,Whatcouldgowrong,7614532,The best place to learn what not to do.,Whatcouldgowrong
1,listentothis,17929902,The musical community of reddit -- Now reopene...,listentothis
2,JusticeServed,1929183,https://archive.is/K6PxR,JusticeServed
3,LeopardsAteMyFace,1311460,"'I never thought leopards would eat MY face,' ...",LeopardsAteMyFace
4,UpliftingNews,20317730,A place to read and share positive and uplifti...,UpliftingNews
...,...,...,...,...
85,IAmA,22474087,"I Am A, where the mundane becomes fascinating ...",IAmA
86,memes,35471936,Memes!\n\nA way of describing cultural informa...,memes
87,insanepeoplefacebook,2046362,Insane people on social media. Post it here.,insanepeoplefacebook
88,aww,37666788,"Things that make you go AWW! -- like puppies, ...",aww


In [79]:
subreddit['name'] = subreddit['name'].str.lower()
subreddit['description'] = subreddit['description'].str.lower()
subreddit['subreddit'] = subreddit['subreddit'].str.lower()

In [84]:
# Función segura para tokenizar texto
def safe_tokenize(text):
    if isinstance(text, str):
        return tokenizer.encode(text, add_special_tokens=True)
    return []

# tokenizar columnas 'name', 'description' y 'subreddit
subreddit['name_tokenized'] = subreddit['name'].apply(safe_tokenize)
subreddit['description_tokenized'] = subreddit['description'].apply(safe_tokenize)
subreddit['subreddit_tokenized'] = subreddit['subreddit'].apply(safe_tokenize)

In [85]:
subreddit

,name,subscribers,description,subreddit,is_subscribed,name_tokenized,description_tokenized,subreddit_tokenized
0,whatcouldgowrong,7614532,the best place to learn what not to do.,whatcouldgowrong,False,"[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]","[101, 1996, 2190, 2173, 2000, 4553, 2054, 2025...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]"
1,listentothis,17929902,the musical community of reddit -- now reopene...,listentothis,False,"[101, 4952, 3406, 15222, 2015, 102]","[101, 1996, 3315, 2451, 1997, 2417, 23194, 101...","[101, 4952, 3406, 15222, 2015, 102]"
2,justiceserved,1929183,https://archive.is/k6pxr,justiceserved,False,"[101, 19867, 25944, 102]","[101, 16770, 1024, 1013, 1013, 8756, 1012, 200...","[101, 19867, 25944, 102]"
3,leopardsatemyface,1311460,"'i never thought leopards would eat my face,' ...",leopardsatemyface,False,"[101, 16240, 16846, 26662, 12172, 102]","[101, 1005, 1045, 2196, 2245, 16240, 2015, 205...","[101, 16240, 16846, 26662, 12172, 102]"
4,upliftingnews,20317730,a place to read and share positive and uplifti...,upliftingnews,False,"[101, 2039, 26644, 2638, 9333, 102]","[101, 1037, 2173, 2000, 3191, 1998, 3745, 3893...","[101, 2039, 26644, 2638, 9333, 102]"
...,...,...,...,...,...,...,...,...
85,iama,22474087,"i am a, where the mundane becomes fascinating ...",iama,False,"[101, 24264, 2863, 102]","[101, 1045, 2572, 1037, 1010, 2073, 1996, 2468...","[101, 24264, 2863, 102]"
86,memes,35471936,memes!\n\na way of describing cultural informa...,memes,False,"[101, 2033, 7834, 102]","[101, 2033, 7834, 999, 1037, 2126, 1997, 7851,...","[101, 2033, 7834, 102]"
87,insanepeoplefacebook,2046362,insane people on social media. post it here.,insanepeoplefacebook,False,"[101, 9577, 5051, 27469, 12172, 8654, 102]","[101, 9577, 2111, 2006, 2591, 2865, 1012, 2695...","[101, 9577, 5051, 27469, 12172, 8654, 102]"
88,aww,37666788,"things that make you go aww! -- like puppies, ...",aww,False,"[101, 22091, 2860, 102]","[101, 2477, 2008, 2191, 2017, 2175, 22091, 286...","[101, 22091, 2860, 102]"


In [ ]:
#top_posts
top_post = pd.read_csv('/Users/isavalladolid/Documents/Documentos - MacBook Air de Isabel/IMEDIA_Project/data/interim/top_posts_data.csv')
top_post

,title,score,url,subreddit
0,WCGW using escalator as conveyor belt?,222299,https://v.redd.it/9i43f3tdcel91,Whatcouldgowrong
1,WCGW when you ask a fashion blogger a nuclear ...,164017,https://v.redd.it/so7rra8506a91,Whatcouldgowrong
2,what could go wrong harassing a muslim family?,162500,https://i.redd.it/cebjpod17at51.jpg,Whatcouldgowrong
3,WCGW trying to copy a technique without planning?,143302,https://v.redd.it/90sip090acha1,Whatcouldgowrong
4,WCGW Forgetting to switch to the Alt account,141633,https://i.redd.it/ei2u7pok8hy51.jpg,Whatcouldgowrong
...,...,...,...,...
8995,Ran into these two at Blue Jays game.,57105,https://imgur.com/moYKfI9,DunderMifflin
8996,Too many ads...,57087,https://i.redd.it/e25o6oz901961.jpg,DunderMifflin
8997,The last couple of weeks have been wild,56964,https://i.imgur.com/kAhVjdC.jpg,DunderMifflin
8998,When my wife sends the kids out to play in the...,56599,https://i.imgur.com/No0UzvH.gifv,DunderMifflin


In [45]:
top_post['title'] = top_post['title'].str.lower()
top_post['subreddit'] = top_post['subreddit'].str.lower()

In [86]:
# Función segura para tokenizar texto
def safe_tokenize(text):
    if isinstance(text, str):
        return tokenizer.encode(text, add_special_tokens=True)
    return []

# tokenizar columnas 'title'y 'subreddit'
top_post['title_tokenized'] = top_post['title'].apply(safe_tokenize)
top_post['subreddit_tokenized'] = top_post['subreddit'].apply(safe_tokenize)

In [87]:
top_post

,title,score,url,subreddit,title_tokenized,subreddit_tokenized
0,wcgw using escalator as conveyor belt?,222299,https://v.redd.it/9i43f3tdcel91,whatcouldgowrong,"[101, 15868, 2290, 2860, 2478, 9686, 25015, 42...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]"
1,wcgw when you ask a fashion blogger a nuclear ...,164017,https://v.redd.it/so7rra8506a91,whatcouldgowrong,"[101, 15868, 2290, 2860, 2043, 2017, 3198, 103...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]"
2,what could go wrong harassing a muslim family?,162500,https://i.redd.it/cebjpod17at51.jpg,whatcouldgowrong,"[101, 2054, 2071, 2175, 3308, 18820, 18965, 10...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]"
3,wcgw trying to copy a technique without planning?,143302,https://v.redd.it/90sip090acha1,whatcouldgowrong,"[101, 15868, 2290, 2860, 2667, 2000, 6100, 103...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]"
4,wcgw forgetting to switch to the alt account,141633,https://i.redd.it/ei2u7pok8hy51.jpg,whatcouldgowrong,"[101, 15868, 2290, 2860, 17693, 2000, 6942, 20...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]"
...,...,...,...,...,...,...
8995,ran into these two at blue jays game.,57105,https://imgur.com/moYKfI9,dundermifflin,"[101, 2743, 2046, 2122, 2048, 2012, 2630, 1893...","[101, 24654, 4063, 4328, 4246, 4115, 102]"
8996,too many ads...,57087,https://i.redd.it/e25o6oz901961.jpg,dundermifflin,"[101, 2205, 2116, 14997, 1012, 1012, 1012, 102]","[101, 24654, 4063, 4328, 4246, 4115, 102]"
8997,the last couple of weeks have been wild,56964,https://i.imgur.com/kAhVjdC.jpg,dundermifflin,"[101, 1996, 2197, 3232, 1997, 3134, 2031, 2042...","[101, 24654, 4063, 4328, 4246, 4115, 102]"
8998,when my wife sends the kids out to play in the...,56599,https://i.imgur.com/No0UzvH.gifv,dundermifflin,"[101, 2043, 2026, 2564, 10255, 1996, 4268, 204...","[101, 24654, 4063, 4328, 4246, 4115, 102]"


In [47]:
#ultimos posts
ultimos_post = pd.read_csv('/Users/isavalladolid/Documents/Documentos - MacBook Air de Isabel/IMEDIA_Project/data/interim/ultimos_posts_data.csv')
ultimos_post

,title,score,url,subreddit
0,WCGW throwing blocks of ice in the air,6018,https://v.redd.it/lbovh00qa0ye1,Whatcouldgowrong
1,Something tells me this isn’t the first time h...,3001,https://v.redd.it/ts3kdkvg73ye1,Whatcouldgowrong
2,WCGW if you swin in a tank with a sturgeon,263,https://v.redd.it/sdkb556mk3ye1,Whatcouldgowrong
3,Penfriend -- House Of Stories [indie rock] (2025),11,https://penfriendrocks.bandcamp.com/track/hous...,listentothis
4,Tonnerre - Might [French electronic music] (2023),2,https://www.youtube.com/watch?v=KBj1265JeY4,listentothis
...,...,...,...,...
2951,That's as clear as I can make it. (From New Am...,10,https://i.redd.it/yke6fp8a03ye1.jpeg,DunderMifflin
2952,Do you think Dwight would’ve kept Angela’s cats?,4,https://www.reddit.com/r/DunderMifflin/comment...,DunderMifflin
2953,S4 E11 Survivor Man Foreshadowing,1,https://www.reddit.com/r/DunderMifflin/comment...,DunderMifflin
2954,Are Holly and Michael's kids really named afte...,0,https://www.reddit.com/r/DunderMifflin/comment...,DunderMifflin


In [48]:
ultimos_post['title'] = ultimos_post['title'].str.lower()
ultimos_post['subreddit'] = ultimos_post['subreddit'].str.lower()

In [88]:
# Función segura para tokenizar texto
def safe_tokenize(text):
    if isinstance(text, str):
        return tokenizer.encode(text, add_special_tokens=True)
    return []

# tokenizar columnas 'title'y 'subreddit'
ultimos_post['title_tokenized'] = ultimos_post['title'].apply(safe_tokenize)
ultimos_post['subreddit_tokenized'] = ultimos_post['subreddit'].apply(safe_tokenize)

In [89]:
ultimos_post

,title,score,url,subreddit,title_tokenized,subreddit_tokenized
0,wcgw throwing blocks of ice in the air,6018,https://v.redd.it/lbovh00qa0ye1,whatcouldgowrong,"[101, 15868, 2290, 2860, 6886, 5991, 1997, 325...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]"
1,something tells me this isn’t the first time h...,3001,https://v.redd.it/ts3kdkvg73ye1,whatcouldgowrong,"[101, 2242, 4136, 2033, 2023, 3475, 1521, 1056...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]"
2,wcgw if you swin in a tank with a sturgeon,263,https://v.redd.it/sdkb556mk3ye1,whatcouldgowrong,"[101, 15868, 2290, 2860, 2065, 2017, 25430, 23...","[101, 2054, 3597, 21285, 3995, 13088, 5063, 102]"
3,penfriend -- house of stories [indie rock] (2025),11,https://penfriendrocks.bandcamp.com/track/hous...,listentothis,"[101, 7279, 19699, 9013, 2094, 1011, 1011, 216...","[101, 4952, 3406, 15222, 2015, 102]"
4,tonnerre - might [french electronic music] (2023),2,https://www.youtube.com/watch?v=KBj1265JeY4,listentothis,"[101, 10228, 3678, 2890, 1011, 2453, 1031, 241...","[101, 4952, 3406, 15222, 2015, 102]"
...,...,...,...,...,...,...
2951,that's as clear as i can make it. (from new am...,10,https://i.redd.it/yke6fp8a03ye1.jpeg,dundermifflin,"[101, 2008, 1005, 1055, 2004, 3154, 2004, 1045...","[101, 24654, 4063, 4328, 4246, 4115, 102]"
2952,do you think dwight would’ve kept angela’s cats?,4,https://www.reddit.com/r/DunderMifflin/comment...,dundermifflin,"[101, 2079, 2017, 2228, 14304, 2052, 1521, 231...","[101, 24654, 4063, 4328, 4246, 4115, 102]"
2953,s4 e11 survivor man foreshadowing,1,https://www.reddit.com/r/DunderMifflin/comment...,dundermifflin,"[101, 1055, 2549, 1041, 14526, 12084, 2158, 18...","[101, 24654, 4063, 4328, 4246, 4115, 102]"
2954,are holly and michael's kids really named afte...,0,https://www.reddit.com/r/DunderMifflin/comment...,dundermifflin,"[101, 2024, 9079, 1998, 2745, 1005, 1055, 4268...","[101, 24654, 4063, 4328, 4246, 4115, 102]"
